# Actividad Práctica: Introducción a imágenes medicas

## Objetivo

En este ejercicio práctico trabajaremos con el dataset [Structural MRI Datasets](https://www.kaggle.com/datasets/ilknuricke/neurohackinginrimages) en la plataforma Kaggle. Este conjunto de datos incluye imágenes médicas en formato DICOM (Digital Imaging and Communications in Medicine), ampliamente utilizado en entornos clínicos para almacenar estudios como rayos X, resonancias magnéticas y tomografías.

Al finalizar este taller, el estudiante estará en capacitado para:
* Leer imágenes médicas en formato DICOM utilizando pydicom.
* Acceder a metadatos clínicos y técnicos contenidos en la cabecera DICOM.
* Visualizar imágenes en escala de grises y con diferentes mapas de color (cmap).
* Normalizar y manipular las intensidades de la imagen para facilitar su análisis.
* Aplicar una segmentación binaria simple por umbralización.
* Anonimizar imágenes eliminando o sobrescribiendo datos sensibles.
* Exportar imágenes médicas a formatos estándar (PNG, JPG) preservando la información relevante.


## 1. Introducción al dataset

### 1.1 Descripción del dataset

Este conjunto de datos contiene imágenes cerebrales estructurales obtenidas mediante resonancia magnética (MRI), almacenadas en dos formatos comunes: DICOM y NIfTI. Estos formatos permiten tanto la visualización de las imágenes como un análisis más profundo a través de herramientas de procesamiento avanzadas. El dataset es ideal para aquellos interesados en explorar técnicas de visualización, segmentación, y análisis cuantitativo en neurociencia computacional, especialmente en el contexto de estudios clínicos y de investigación.

#### DICOM (Digital Imaging and Communications in Medicine)

Este es el formato estándar para el almacenamiento y transmisión de imágenes médicas en el ámbito clínico. Los archivos DICOM contienen no solo la imagen en sí, sino también una gran cantidad de metadatos asociados, incluyendo:

- Parámetros técnicos de adquisición (ej. resolución, grosor de corte)  
- Identificadores del paciente (anonimizados en este caso)  
- Fecha y hora del estudio  
- Modalidad de la imagen (ej. T1, T2, FLAIR)  
- Información del dispositivo de escaneo  

En este dataset, las imágenes DICOM están organizadas según su modalidad dentro de subdirectorios:

- **T1, T2, FLAIR**: diferentes secuencias de imagen utilizadas para resaltar distintas características anatómicas.  
- **ROI**: máscaras de regiones de interés generadas manualmente o automáticamente.  

---

#### NIfTI (Neuroimaging Informatics Technology Initiative)

Este es un formato comúnmente utilizado para el análisis e investigación con imágenes médicas. Es especialmente útil en contextos computacionales y cuantitativos, ya que:

- Contiene la imagen en una estructura 3D o 4D  
- Incluye información espacial como la orientación anatómica y el tamaño del voxel  
- Es fácilmente manipulable en software de análisis de imágenes médicas como FSL, ANTs, SPM, o usando librerías como `nibabel` en Python  

Las imágenes en formato NIfTI (.nii o .nii.gz) permiten operaciones como:

- Visualización tridimensional  
- Registro entre imágenes  
- Segmentación de estructuras  
- Extracción de métricas volumétricas o de intensidad  

---

En resumen, el dataset proporciona imágenes médicas estructurales en formatos DICOM y NIfTI, lo que permite tanto la exploración visual de los datos originales como el análisis computacional.

Al descomprimir el archivo ZIP del dataset, se obtiene una estructura de carpetas que incluye múltiples tipos de imágenes estructurales cerebrales. El contenido se organiza de la siguiente manera:

```
BRAINIX
    ├── DICOM
    │   ├── FLAIR
    │   ├── ROI
    │   ├── T1
    │   └── T2
    └── NIFTI

```

### 1.2 Acceso al dataset

Al igual que el taller anterior, el acceso al dataset se realiza a través de la plataforma Kaggle


In [ ]:
#!rm -rf Neurohacking_data-0.0/
#!rm -rf __MACOSX/
#!rm -rf mri_old/
#!rm -rf sample_data/

In [ ]:
!ls

In [ ]:
import kagglehub
import os

# Download latest version
folder = kagglehub.dataset_download("ilknuricke/neurohackinginrimages", force_download=True)

print("Ruta a la carpeta:", folder)

!ln -s {folder} ./mri
!unzip -o mri/Neurohacking_data-0.0.zip

> El símbolo ! en Google Colab (y notebooks en general) permite ejecutar comandos de consola (shell/bash) directamente desde una celda de Python.
> Por ejemplo:
> - !ln -s ... crea un enlace simbólico (acceso directo) a una carpeta.
> - !unzip ... descomprime archivos ZIP usando la utilidad del sistema.
## 2. PyDICOM

[PyDICOM](https://pydicom.github.io/) es una de las librerías más populares para trabajar con archivos DICOM en Python. Con PyDICOM puedes:

* **Cargar archivos DICOM desde un directorio.** :Esto permite leer de forma sencilla cualquier archivo DICOM para luego extraer su contenido.

* **Acceder a los metadatos de la cabecera DICOM.** : La cabecera DICOM contiene información importante (como Patient ID, Patient Age, Modality, etc.) que se puede utilizar para identificar o clasificar las imágenes. PyDICOM facilita la extracción de estos atributos.

* **Modificar los campos de metadatos si es necesario.**: Esto es especialmente útil para la anonimización, ya que puedes sobrescribir o eliminar la información sensible del paciente.

### 2.1 Instalación de PyDICOM
Para instalar PyDICOM, puedes ejecutar el siguiente comando en tu terminal o en una celda de Jupyter:
  
    !pip install pydicom
  
Una vez instalada, podrás utilizar sus funciones para cargar y manipular archivos DICOM.

## 3. Importar librerías necesarias





In [ ]:
!pip install pydicom

In [ ]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt

## 4. Trabajando con imagenes DICOM


### 4.1 Trabajando con multiples archivos
Para listar los archivos DICOM en un directorio específico, puede usar la librería `os` de Python. Esto te permitirá ver qué archivos están disponibles para su procesamiento.

#### **Ejercicio 1**
En este primer ejercicio, se le pide que enumere todos los archivos DICOM (extensión .dcm) presentes en la carpeta  `Neurohacking_data-0.0/BRAINIX/DICOM/` para T1 y T2. Al listar sus nombres, podrá confirmar cuántos archivos hay y tener una visión inicial de los datos disponibles. HINT: Los archivos DICOM tienen la extensión `.dcm`. Puedes usar `os.listdir()` para listar los archivos en un directorio y luego filtrar por estas extensiones.


In [ ]:
# <CODE> utilice listdir aquí para poder acceder a los nombres de las carpetas para T1


In [ ]:
# <CODE> utilice listdir aquí para poder acceder a los nombres de las carpetas para T2



### 4.2 Carga de archivos dicom
En este ejercicio se le pedirá cargar una imagen T1. Primero necesitará importar la librería `pydicom` y luego usar la función `dcmread()` para leer el archivo. Debe asegurarse de que el archivo DICOM esté en el directorio correcto.

```python
import pydicom

# Cargar un archivo DICOM
dicom_file = pydicom.dcmread('ruta/al/archivo.dcm')
```
#### **Ejercicio 2**


En este ejercicio, debe cargar el primer archivo DICOM de la carpeta T1 y T2. Para ello, puede usar la función `dcmread()` de la librería `pydicom`. Debe asegurarse de que el archivo esté en el directorio correcto. Desarrolle el codigo de modo que sea capaz de cargar cualquier archivo DICOM, cambiando el indice del archivo.



In [ ]:
# <CODE> cargue archivos dicom para  T1

In [ ]:
# <CODE> cargue archivos dicom para  T2

## 4.3 Exploración de metadatos

#### **Ejercicio 3**

Revise la cabecera del archivo DCM cargado, para conocer datos como nombre del paciente, modalidad de la imagen, fecha de estudio, tamaño, pixel spacing y protocolo de adquisición. Identifique cuál de estos campos podría considerarse sensible. Puede explorar el header del archivo DICOM con el siguiente comando:
```python
print(dicom_file)
```
o
```python
dicom_file[0x0010, 0x0010].value  # nombre del paciente
```
Donde `0x0010, 0x0010` es el tag del nombre del paciente. Puede encontrar una lista de tags DICOM en la [documentación oficial](https://dicom.innolitics.com/ciods/part10/sect6.2).




In [ ]:
# <CODE> Analice la información en la cabecera dicom de las imágenes de T1 y T2
# PatientName, PatientID, Modality, StudyDate, ProtocolName, Rows, Columns, PixelSpacing


#### **Ejercicio 4**

Para consolidar la información obtenida, recorra todos los archivos .dcm T1 y T2. Extraiga información clave (por ejemplo, modalidad, tamaño de la imagen y fecha del estudio). Almacene estos datos en un DataFrame de pandas que le permita manejar la información de forma tabular.


In [ ]:
# <CODE> Recorra todas las imagenes y genere el data frame para los valores que
# se solicitan.

## 5. Visualización de imágenes DICOM
Para visualizar imágenes DICOM, puede usar la librería `matplotlib`. Debe asegurarse de que la imagen esté en formato de matriz NumPy antes de intentar mostrarla. Puede usar `dicom_file.pixel_array` para acceder a la matriz de píxeles.

#### **Ejercicio 5**
Visualice la imagen DICOM cargada en el ejercicio 2. Asegurese  de que la imagen se muestre correctamente y ajuste el mapa de color (cmap) si es necesario. Puede usar `plt.imshow()` para mostrar la imagen y `plt.axis('off')` para ocultar los ejes.




In [ ]:
# <CODE> Muestre una imagen DICOM


### 5.1 Mapa de color

Un mapa de color (cmap) es una representación visual que asigna colores a diferentes valores de intensidad en una imagen. En el contexto de imágenes médicas, los mapas de color pueden ayudar a resaltar características específicas de la imagen, facilitando su interpretación.
Los mapas de color son especialmente útiles en imágenes en escala de grises, donde los valores de intensidad pueden no ser fácilmente interpretables. Al aplicar un cmap, puede transformar la imagen en una representación más intuitiva y visualmente atractiva.

Para aplicar un cmap a una imagen, puede usar la función `imshow(cmap='cmap')` de `matplotlib` y especificar el cmap deseado. Algunos cmap comunes son:
- `gray`: Escala de grises
- `hot`: Mapa de calor
- `jet`: Mapa de colores en espectro
- `viridis`: Mapa de colores perceptualmente uniforme
- `plasma`: Mapa de colores perceptualmente uniforme


#### **Ejercicio 6**
Aplique diferentes mapas de color a la imagen DICOM cargada en el ejercicio 2. Experimente con al menos tres cmap diferentes y observe cómo cambia la visualización de la imagen. Puedes usar `plt.imshow()` para mostrar la imagen y `plt.axis('off')` para ocultar los ejes.


In [ ]:
# <CODE> Muestre una imagen DICOM con distinos mapas de colores


### 5.2 Visualización de multiple imágenes

Para visualizar múltiples imágenes DICOM en una sola figura, puede usar subgráficas (subplots) de `matplotlib`. Esto es útil para comparar diferentes imágenes o diferentes cortes de la misma imagen. La sintaxis básica para crear subgráficas es la siguiente:


```python
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 10))
plt.subplot(2, 2, 1)  # Fila 1, Columna 1
plt.imshow(image1, cmap='gray')
plt.axis('off')
plt.subplot(2, 2, 2)  # Fila 1, Columna 2
plt.imshow(image2, cmap='gray')
plt.axis('off')
plt.subplot(2, 2, 3)  # Fila 2, Columna 1
plt.imshow(image3, cmap='gray')
plt.axis('off')
plt.subplot(2, 2, 4)  # Fila 2, Columna 2
plt.imshow(image4, cmap='gray')
plt.axis('off')
plt.show()


```

O otra alternativa es la siguiente

```python
fig, ax = plt.subplots(2,2,figsize=(10, 10))
ax[0,0].imshow(image1, cmap='gray') # Fila 1, Columna 1
ax[0,0].axis('off')
ax[0,1].imshow(image2, cmap='gray') # Fila 1, Columna 2
ax[0,1].axis('off')
ax[1,0].imshow(image3, cmap='gray') # Fila 2, Columna 1
ax[1,0].axis('off')
ax[1,1].imshow(image4, cmap='gray') # Fila 2, Columna 2
ax[1,1].axis('off')
```

#### **Ejercicio 7**

Selecciona 4 archivos al azar de tu lista DICOM y crea un subplot de 2x2 para mostrarlos para imagenes T1 y T2. En cada imagen, coloca un título que incluya su modalidad y fecha de estudio. Asegúrate de que todas las imágenes tengan el mismo tamaño y escala de color para facilitar la comparación visual. Puedes usar `plt.title()` para agregar títulos a cada subgráfica.




In [ ]:
# <CODE> muestre el subplot para T1

In [ ]:
# <CODE> muestre el subplot para T2

### 5.3 Normalización de la imagen

La normalización de imágenes es un proceso que ajusta los valores de intensidad de los píxeles para que estén dentro de un rango específico. Esto es útil para mejorar el contraste y la visibilidad de características específicas en la imagen. La normalización puede ser especialmente importante en imágenes médicas, donde las intensidades pueden variar significativamente entre diferentes estudios o modalidades.

#### **Ejercicio 8**
Normalice la imagen DICOM cargada en el ejercicio 2. Puede usar la siguiente fórmula para normalizar los valores de intensidad:
```python
normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))
```

Compare el histograma de la imagen original con el de la imagen normalizada, puede convertir el arreglo 2D a un arreglo 1D usando los metodos de numpy `.ravel()` o `.flatten()`. Puede usar `plt.hist()` para mostrar los histogramas y `plt.show()` para visualizarlos.





In [ ]:
# <CODE> muestre la imagen sin normalizar y normalizada, junto a sus histogramas


### 5.3 Ajuste de brillo y contraste
El ajuste de brillo y contraste es un proceso que modifica la intensidad de los píxeles en una imagen para mejorar su visibilidad. Esto es especialmente útil en imágenes médicas, donde ciertas estructuras pueden no ser claramente visibles debido a la variabilidad en la adquisición de imágenes.

El brillo se refiere a la cantidad de luz en la imagen, mientras que el contraste se refiere a la diferencia entre las áreas más claras y más oscuras de la imagen. Ajustar ambos parámetros puede ayudar a resaltar características específicas y facilitar el diagnóstico.

En particular , el ajuste de contraste puede ser útil para resaltar estructuras anatómicas o lesiones que de otro modo podrían pasar desapercibidas. Esto se logra al aumentar la diferencia entre los valores de intensidad de los píxeles, lo que hace que las áreas oscuras sean más oscuras y las áreas claras sean más claras. La formula básica para ajustar el contraste es:

$$I_{out} = I^\gamma$$

Donde $I_{out}$ es la imagen de salida, $I$ es la imagen de entrada e $\gamma$ es el factor gamma que controla el contraste. Un valor de $g$ mayor que 1 aumentará el contraste, mientras que un valor menor que 1 lo disminuirá.



#### **Ejercicio 9**
Ajusta el brillo y contraste de la imagen DICOM cargada en el ejercicio 2. Utiliza la fórmula mencionada anteriormente para ajustar el contraste. Experimenta con diferentes valores de gamma y observa cómo afecta la visualización de la imagen.


In [ ]:
# <CODE> muestre la imagen original y con ajuste de brillo, junto a sus histogramas


## 6. Segmentación de imágenes

### 6.1 Umbralización

La segmentación de imágenes es un proceso que implica dividir una imagen en diferentes regiones o segmentos, cada uno de los cuales representa una característica o estructura específica. En el contexto de imágenes médicas, la segmentación es crucial para identificar y analizar estructuras anatómicas, lesiones o patologías.

La segmentación puede ser realizada de varias maneras, incluyendo técnicas basadas en umbralización, agrupamiento, y aprendizaje automático. En este ejercicio, nos centraremos en la segmentación por umbralización, que es una de las técnicas más simples para segmentar imágenes médicas.

El proceso de segmentación por umbralización implica establecer un valor de umbral y clasificar los píxeles de la imagen en dos categorías: aquellos que están por encima del umbral y aquellos que están por debajo.

Esto se puede realizar utilizando cualquier numpy array, y el resultado es una imagen binaria donde los píxeles que cumplen la condición se establecen en 1  y los demás en 0. Ejemplo:
```python
import numpy as np
import matplotlib.pyplot as plt

imagen = np.array([[0, 50, 100], [150, 200, 255]])
umbral = 100
mask = imagen > umbral
segmentada = imagen * mask
print(segmentada)
```

#### **Ejercicio 10**
Realice una segmentación binaria de la imagen DICOM cargada en el ejercicio 2 utilizando un valor de umbral fijo. La segmentación debe entregar como resultado la materia blanca del cerebro. Puede utilizar la imagen normalizada como entrada y aplicar correcciones de contraste si es necesario. Visualice la imagen original, su mascara y el resultado de la segmentación en un subplot de 1x3.



In [ ]:
# <CODE> muestre la imagen original, segmentacion y la aplicacion de la
# segmentacion sobre la imagen original


### 6.2  Mascara sombreada
La segmentación de imágenes médicas a menudo implica la creación de máscaras que resaltan áreas específicas de interés. Una técnica común es aplicar una máscara sombreada, que combina la imagen original con una máscara binaria para resaltar las áreas segmentadas.

#### **Ejercicio 11**
Aplique una máscara sombreada a la imagen DICOM cargada en el ejercicio 2 utilizando la máscara generada en el ejercicio 10. La máscara sombreada debe resaltar las áreas segmentadas en la imagen original. Visualice la imagen original, la máscara y la imagen con la máscara aplicada en un subplot de 1x3. Puede usar `plt.imshow()` para mostrar las imágenes y `plt.axis('off')` para ocultar los ejes.


In [ ]:
# <CODE> muestre la imagen original, imagen segmentada y enmascarada


## 7. Guardado de imágenes

### 7.1 Guardado de imagenes en formato DICOM

Para guardar imágenes DICOM, puedes usar la función `save_as()` de la librería `pydicom`. Esto le permite guardar la imagen y los metadatos en un nuevo archivo DICOM. Por otra parte, es posible anonimizar la imagen eliminando o sobrescribiendo datos sensibles. Esto es especialmente importante en el contexto de imágenes médicas, donde la privacidad del paciente es fundamental.

Por ejemplo, puede eliminar el nombre del paciente y otros identificadores personales antes de guardar la imagen. Esto se puede hacer modificando directamente los atributos de la cabecera DICOM.
```python

dicom_file.PatientName = "Anonymous"
dicom_file.PatientID = "000000"

# o
dicom_file[0x0010, 0x0010].value = "Anonymous"
```

#### **Ejercicio 12**
Guarde la imagen DICOM cargada en el ejercicio 2 como un nuevo archivo DICOM. Asegurándose de anonimizar la imagen eliminando o sobrescribiendo los datos sensibles del paciente. Puede usar la función `save_as()` de `pydicom` para guardar la imagen y los metadatos en un nuevo archivo DICOM.



In [ ]:
# <CODE> Genere un codigo que guarde unas de las imagenes y anonimice los datos
# sensibles.


#### **Ejercicio 13**

Carga el archivo anonimizado y comprueba que los campos sensibles queden en blanco o contengan valores genéricos.


In [ ]:
# <CODE> Cargue el archivo que guardo y verifique que este anonimizada
# correctamente.


### 7.2 Guardado de imagenes en formato PNG
Para guardar imágenes en formato PNG, puedes usar la librería `PIL` (Python Imaging Library). Guardar la imagen en formato PNG le permite compartirla fácilmente y utilizarla en otros contextos. No obstante, es importante tener en cuenta que al guardar la imagen en un formato diferente, es posible que se pierdan gran parte de los metadatos y que la información de la imagen no sea completamente preservada.

Las imagenes se deben formatear de manera correcta para que sean guardadas en el formato deseado. En este caso, la imagen debe ser convertida a un formato compatible con PNG. Esto se puede hacer utilizando la función `convert()` de `PIL`. Por ejemplo, si la imagen es una matriz NumPy, puede convertirla a un objeto PIL de la siguiente manera:

```python
from PIL import Image
# Convertir la matriz NumPy a un objeto PIL
image_pil = Image.fromarray(normalized_image.astype(np.uint8))
```
Luego, puede guardar la imagen en formato PNG utilizando la función `save()` de `PIL`:
```python
image_pil.save('ruta/al/archivo.png')
```


#### **Ejercicio 14**
Guarde la imagen DICOM cargada en el ejercicio 2 como un nuevo archivo PNG. Genere 2 archivos PNG: uno con la imagen original y otro con la imagen normalizada. Comente las diferencias que se observan.


In [ ]:
# <CODE> Gaurde la imagen en archivo .png, imagen original e imagen normalizada


___________________________________

# PROYECTO FINAL MODULO 1 DIADM

Fecha de entrega miercoles 31 de octubre del 2025.

**GRUPO XX**

<font color='red'>Reemplace "XX" por el número del grupo, y elimine esta línea de texto cuando entregue su trabajo</font>

____________________________________

### **Relevancia de la preparación de datos para Inteligencia Artificial en Imágenes Médicas**

En el desarrollo de soluciones de inteligencia artificial (IA) aplicadas al diagnóstico médico por imágenes, un paso fundamental —y muchas veces subestimado— es la preparación y exploración de los datos. La calidad y consistencia del conjunto de datos de entrenamiento puede afectar directamente el rendimiento, la generalización y la confiabilidad de los modelos de IA.

En este proyecto, se le asignará la primera etapa crítica del pipeline de un sistema de IA: la preparación y exploración de un conjunto de datos basado en imágenes médicas en formato DICOM. Este conjunto será eventualmente utilizado predecir la gravedad del deterioro de la función pulmonar de un paciente a partir de una tomografía computarizada (CT) y test de capacidad pulmonar.


### **Contexto clínico**

Imagina que, de un día para otro, tu respiración se vuelve constantemente dificultosa y superficial. Meses después, finalmente te diagnostican fibrosis pulmonar, un trastorno sin causa conocida y sin cura, provocado por cicatrices en los pulmones. Si eso te ocurriera, querrías saber cuál es tu pronóstico. Es en ese momento cuando una enfermedad preocupante se vuelve aterradora para el paciente: los resultados pueden variar desde una estabilidad a largo plazo hasta un deterioro rápido, pero los médicos no pueden determinar fácilmente en qué punto de ese espectro se encuentra cada persona. Tu ayuda, junto con la ciencia de datos, podría contribuir en esta predicción, lo que sería de gran ayuda tanto para los pacientes como para los profesionales de la salud.

Los métodos actuales dificultan el tratamiento de enfermedades pulmonares fibróticas, incluso teniendo acceso a tomografías computarizadas (CT) de tórax. Además, la amplia variabilidad en los pronósticos complica la organización de ensayos clínicos. Finalmente, los pacientes experimentan una ansiedad extrema —además de los síntomas propios de la fibrosis— debido al incierto curso de la enfermedad.

El Open Source Imaging Consortium (OSIC) es una iniciativa cooperativa sin fines de lucro entre el mundo académico, la industria y organizaciones filantrópicas. El grupo permite avances rápidos en la lucha contra la Fibrosis Pulmonar Idiopática (IPF), las enfermedades pulmonares intersticiales fibróticas (ILDs) y otras enfermedades respiratorias, incluidas las condiciones enfisematosas. Su misión es reunir a radiólogos, médicos y científicos computacionales de todo el mundo para mejorar los tratamientos basados en imágenes.

El desafío posterior a la preparación de datos, consiste en utilizar técnicas de aprendizaje automático para realizar predicciones utilizando como entrada la imagen, los metadatos y la FVC (capacidad vital forzada) basal. Se determinará la función pulmonar en base a los resultados de un espirómetro, el cual mide el volumen de aire inhalado y exhalado.

Si se tiene éxito, los pacientes y sus familias podrían comprender mejor su pronóstico al momento del diagnóstico de esta enfermedad pulmonar incurable. Una mejor detección de la gravedad también tendría un impacto positivo en el diseño de ensayos clínicos y aceleraría el desarrollo clínico de nuevos tratamientos.

### **Descripción del Conjunto de Datos**

El conjunton de dato fue extraido de la siguiente base de datos: https://www.kaggle.com/competitions/osic-pulmonary-fibrosis-progression/data. (**Adicionalmente pueden descargarlo del siguiente link de Google Drive:** https://drive.google.com/file/d/1r-I60Z55xXTHFCiCvIMsrajKdN53JnhE/view?usp=sharing )

En el conjunto de datos se proporciona una tomografía basal de tórax y la información clínica asociada para un grupo de pacientes. Cada paciente tiene una imagen adquirida en la semana 0 y múltiples visitas de seguimiento a lo largo de aproximadamente 1 a 2 años, en las cuales se mide su FVC.

En el conjunto de prueba se proporciona solo una tomografía basal y la medición inicial de FVC.

Hay aproximadamente 200 casos en total, combinando los conjuntos de prueba público y privado. Esta división es aproximadamente 15% público y 85% privado.

Dado que se trata de datos médicos reales, notarás que el tiempo relativo de las mediciones de FVC varía considerablemente. El momento de la medición inicial con respecto a la CT basal y la duración hasta los puntos de predicción puede ser diferente para cada paciente.

Los archivos y carpetas que usaremos en este proyecto son los siguientes:

**Archivos**

* `train.csv` - conjunto de entrenamiento, contiene el historial completo de información clínica

* `test.csv` - conjunto de prueba, contiene solo la medición basal

* `train/` - carpeta con las tomografías basales en formato DICOM de los pacientes del conjunto de entrenamiento

* `test/` - carpeta con las tomografías basales en formato DICOM de los pacientes del conjunto de prueba


**Columnas**

train.csv y test.csv

* `Patient`: identificador único de cada paciente (también es el nombre de la carpeta DICOM del paciente)

* `Weeks`: número de semanas relativas respecto a la CT basal (puede ser negativo)

* `FVC`: capacidad pulmonar registrada, en mililitros

* `Percent`: campo calculado que aproxima el FVC del paciente como un porcentaje del FVC típico para una persona con características similares

* `Age`: edad

* `Sex`: sexo

* `SmokingStatus`: historial de tabaquismo


In [ ]:
# Monte su Google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Descomprimir la carpeta que esta en el google drive, y la esta guardando en el mismo directorio de google drive
# !unzip '/content/drive/MyDrive/DIADM_2025/DIADM 2025 - GOOGLECOLAB - V2 - MODULO 1/CODIGOS_SEMANA6_18102025/DATA_PROYECTO_FINAL/osic-pulmonary-fibrosis-progression.zip' -d '/content/drive/MyDrive/DIADM_2025/DIADM 2025 - GOOGLECOLAB - V2 - MODULO 1/CODIGOS_SEMANA6_18102025/DATA_PROYECTO_FINAL/'

In [ ]:
# Instalamos los modulos que haya que instalar para poder ejecutar nuestro codigo
!pip install pydicom

In [ ]:
# Importamos los modulos
import os
import cv2
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### **Tareas que deben abordar en este proyecto**

1. **Análisis exploratorio de datos (40% de la nota final).**

* Lea los datos de `train.csv` en un dataframe llamado `base_de_datos`.
* Reponda las siguientes preguntas a partir del dataframe anterior y muestre un gráfico asociado a cada una de ellas.



In [ ]:
# <CODE> lea el achivo train.csv

#### PREGUNTA 1.1

¿ Cuantos pacientes son no-fumadores, ex-fumadores y fumadores?.

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 1.1</font>


#### PREGUNTA 1.2

De los pacientes descritos en el punto anterior ¿ cuantas mujeres y cuantos hombres hay en cada uno de los grupos?.

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 1.2</font>

#### PREGUNTA 1.3

Separe las edades de los pacientes en 3 grupos del mismo rango, ¿cuantos pacientes hay en cada uno de los grupos?.
    
    

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 1.3</font>

#### PREGUNTA 1.4

Agrupe la primera y ultima medición de FVC para cada uno de los pacientes, seperados por estatus de fumador, calcule su promedio y desviacion estandar. ¿ cual es el grupo que presenta mayor deterioro en la FCV, entre la primera y ultima medición ?

    

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 1.4</font>

#### PREGUNTA 1.5

¿ Cuantas semanas en promedio existen entre la primera y ultima medicion de FCV entre todos los pacientes ?.

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 1.5</font>

2. **Análisis de imágenes DICOM (50% de la nota final).**

* Genere un dataframe llamado `base_de_imagenes`, a partir de la información contenida en las carpetas `train/`, con las siguientes columnas:

  * `Patient`: identificador único de cada paciente (también es el nombre de la carpeta DICOM del paciente).
  * `Carpeta`: identificador que indica el nombre de la carpeta train o test.
  * `Rows`: número de filas de la imágen.
  * `Columns`: número de columnas de la imágen.
  * `Num_files`: cantidad de archivos contenidos en cada una de las carpetas.
  * `PixelSpacing_0`: tamaño de pixel en direccion de las filas.
  * `PixelSpacing_1`: tamaño de pixel en direccion de las columnas.
  * `SpacingBetweenSlices`: espaciado entre slices.
  * `SliceThickness`: espezor de slice.
  * `Manufacturer`: marca del equipo donde se adquirieron las imagenes.
  * `KVP`: Pico de kilo voltaje de salida del generador de rayos X utilizado.
  * `XRayTubeCurrent`: Corriente del tubo de rayos X en mA.
  * `GeneratorPower`:	Potencia en kW del generador de rayos X.

* Reponda las siguientes preguntas a partir del dataframe generado y muestre un gráfico asociado a cada una de ellas.



In [ ]:
# <CODE> genere la base_de_imagenes con la informacion contenida en train/

#### PREGUNTA 2.1

¿ Cuantos equipos de CT distintos se utilizaron en la base de datos, y cuantos pacientes hay por cada uno de ellos ?

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 2.1</font>

#### PREGUNTA 2.2
¿ Existe una relacion entre la cantidad de imágenes en cada carpeta y el equipo con el que se adquirieron ?

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 2.2</font>

#### PREGUNTA 2.3

¿ Como es la relación entre KVP y XRayTubeCurrent ?

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 2.3</font>

#### PREGUNTA 2.4
¿ Todos volumenes tienen el mismo tamaño ?.

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

In [ ]:
# <CODE> muestre un grafico.

<font color='red'>RESPONDA ACA LA PREGUNTA 2.4</font>

#### PREGUNTA 2.5
Lea las imágenes de la carpeta test/ y cambie los tamaños de cada una de ellas a 256x256 y guardelas en formato .png, dentro de una nueva carpeta llamada test_png/XXX/, donde XXX corresponde a la subcarpeta cuyo nombre es el ID del paciente. En esta pregunta no debe hacer un grafico.

In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.

3. **Combinación de datos (10% de la nota final).**

Genere un dataframe combinado entre `base_de_datos` y `base_de_imagenes`.


In [ ]:
# <CODE> genere un codigo que permita analizar la pregunta.